<a href="https://colab.research.google.com/github/guilhermefritz/Machine_Learning/blob/main/projeto_custo_do_seguro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Projeto-Prevendo o custo do seguro***

**Problema de Regressão Linear.**

Nesse projeto estarei trabalhando em cima do dataset disponivel no https://www.kaggle.com/jamiemorales/linear-regression-insurance-prices



In [ ]:
#Importando os pacotes que serão utilizados
import pandas as pd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
#importando o dataset 
df=pd.read_csv("/content/insurance.csv")

#Visualizando uma parte dos dados
df.head()



,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
#Puxando informações de cada variavel
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1337 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1337 non-null   int64  
 1   sex       1337 non-null   int64  
 2   bmi       1337 non-null   float64
 3   children  1337 non-null   int64  
 4   smoker    1337 non-null   int64  
 5   region    1337 non-null   object 
 6   charges   1337 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 83.6+ KB


In [ ]:
#verificando se existe algum valor missing  
df.isnull().sum()


age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [ ]:
#verificando se existe algum valor duplicado
df.duplicated().sum()



0

In [ ]:
#mostrando quais valores estao duplicados
df[df.duplicated(keep=False)]


,age,sex,bmi,children,smoker,region,charges


In [ ]:
#eliminando o segundo valor duplicado
df.drop_duplicates(keep='first',inplace=True)

In [ ]:
#verificando se realmente eliminou
df[df.duplicated(keep=False)]

,age,sex,bmi,children,smoker,region,charges


In [ ]:
#Olhando como ficou
df.iloc[195]

age                19
sex              male
bmi             30.59
children            0
smoker             no
region      northwest
charges     1639.5631
Name: 195, dtype: object

In [ ]:
#Olhando como ficou
df.iloc[581]

age                39
sex              male
bmi             45.43
children            2
smoker             no
region      southeast
charges     6356.2707
Name: 582, dtype: object

In [ ]:
#vendo os valores e suas respectivas quantidades dentro da coluna 'sex'(genero)
df.sex.value_counts()


In [ ]:
#Visualizaçao grafica dA
sns.histplot(df.sex)

In [ ]:
#vendo os valores e suas respectivas quantidades dentro da coluna 'smoker'(pessoas fumantes)
df.smoker.value_counts()


no     1063
yes     274
Name: smoker, dtype: int64

In [ ]:
#Explorando a visualização dos dados
sns.pairplot(df)

In [ ]:
#Visualizacao grafica disso
sns.histplot(df.smoker)

In [ ]:
# Parte de treino para a transformação das variaveis categoricas para a utilizaçao no modelo
#Elas são importantes para a construçao do modelo preditivo


le=LabelEncoder()
le.fit(df['sex'].drop_duplicates())
le.fit(df['smoker'].drop_duplicates())

LabelEncoder()

In [ ]:
#Parte de transformar
df.sex=le.fit_transform(df['sex'])
df.smoker=le.fit_transform(df['smoker'])

In [ ]:
#Verificamos agora a variavel 'sex' em sua forma numerica
df.sex.unique()

array([0, 1])

In [ ]:
#Verificamos agora a variavel 'smoker' em sua forma numerica
df.smoker.unique()

array([1, 0])

In [ ]:
#Chamamos a correlaçao das variaveis quando relacionadas a variavel 'charges'(Custo do seguro)
#Verificamos que a variavel 'smoker' é a que mais tem correlação positiva
df.corr()['charges']

<ipython-input-24-25340200184f>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()['charges']


age         0.298308
sex         0.058044
bmi         0.198401
children    0.067389
smoker      0.787234
charges     1.000000
Name: charges, dtype: float64

In [ ]:
#Analise descritiva das variaveis
df.describe()

,age,sex,bmi,children,smoker,charges
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,39.222139,0.504862,30.663452,1.095737,0.204936,13279.121487
std,14.044333,0.500163,6.100468,1.205571,0.403806,12110.359656
min,18.000000,0.000000,15.960000,0.000000,0.000000,1121.873900
25%,27.000000,0.000000,26.290000,0.000000,0.000000,4746.344000
50%,39.000000,1.000000,30.400000,1.000000,0.000000,9386.161300
75%,51.000000,1.000000,34.700000,2.000000,0.000000,16657.717450
max,64.000000,1.000000,53.130000,5.000000,1.000000,63770.428010


Criação dos  Modelos Preditivos 

Parte 1- Regressão Linear Simples

In [ ]:
#Separamos X e y,em que y fica com a variavel target
X=df['smoker'].values.reshape(-1,1)
y=df['charges'].values.reshape(-1,1)

In [ ]:
#Dividimos os dados em  80 % para treino e 20 % para teste
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=7)

In [ ]:
#Criamos o modelo
modelo=LinearRegression()
#Treinamos o modelo
modelo.fit(X_train,y_train)

#Fazemos a predição
modelo_feito=modelo.predict(X_test)
#Agora exibiremos a Acurácia do modelo e embaixo exibiremos o valor real do seguro comparado com o valor predito pelo modelo
real_predito=pd.DataFrame({'real':y_test.flatten(),
                           'predito':modelo_feito.flatten()})

print('Acurácia: %.2f'% (modelo.score(X_test, y_test)*100), '%' )
print(real_predito)

Acurácia: 61.73 %
            real       predito
0    11090.71780   8425.958419
1     9288.02670   8425.958419
2     1694.79640   8425.958419
3     7526.70645   8425.958419
4     4673.39220   8425.958419
..           ...           ...
263   9850.43200   8425.958419
264  10226.28420   8425.958419
265   5012.47100   8425.958419
266   4949.75870   8425.958419
267  18259.21600  31622.980783

[268 rows x 2 columns]


Parte 2- Regressão Linear Múltipla

In [ ]:
#Fazemos um 'drop' no x para a eliminação das variaveis 'charges' e 'region'
#Todas as outras variaveis do dataset estao dentro do X ,menos as 2 que eliminamos
X=df.drop(['charges','region'],axis=1)

#y recebe o target 
y=df['charges'].values.reshape(-1,1)

#Dividimos os dados em  80 % para treino e 20 % para teste
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=7)


In [ ]:
#Criamos o modelo
modelo=LinearRegression()

modelo.fit(X_train,y_train)
#E agora chamamos a Acuracia
print('Acurácia: %.2f'% (modelo.score(X_test, y_test)*100), '%' )
#Acuracia teve uma melhora de 12,67 por cento ao se utilizar varias variaveis para prever

Acurácia: 74.40 %
